In [1]:
import torch
import timm
import numpy as np
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from PIL import Image

# ================================
# 1️⃣ LOAD DATASET
# ================================
dataset_name = "Team-SknAI/SknAI_300_v3_11Labels"
datasets = load_dataset(dataset_name)
datasets = datasets["train"].train_test_split(test_size=0.2, stratify_by_column="label")

# ================================
# 2️⃣ DEFINE TRANSFORMATIONS
# ================================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ================================
# 3️⃣ CUSTOM DATASET CLASS
# ================================
class SkinDataset(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        try:
            img = self.dataset[idx]["image"]
            img = Image.fromarray(np.array(img).astype("uint8")).convert("RGB")
            img = self.transform(img)
            label = torch.tensor(self.dataset[idx]["label"], dtype=torch.long)
            return img, label
        except Exception as e:
            print(f"Error loading image at index {idx}: {e}")
            return torch.zeros(3, 224, 224), torch.tensor(0)

# ================================
# 4️⃣ CREATE DATA LOADERS (Fixed)
# ================================
batch_size = 16

train_dataset = SkinDataset(datasets["train"], transform)
val_dataset = SkinDataset(datasets["test"], transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)  # No multiprocessing
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)  # No multiprocessing

# ================================
# 5️⃣ DEFINE MODEL
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class DenseNet121Model(nn.Module):
    def __init__(self, num_classes):
        super(DenseNet121Model, self).__init__()
        self.model = timm.create_model("densenet121", pretrained=True, num_classes=num_classes)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.model(x)

num_classes = len(datasets["train"].features["label"].names)
model = DenseNet121Model(num_classes).to(device)



Using device: cuda


In [4]:
# ================================
# 6️⃣ TRAINING SETUP (Loss Fix)
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()  # Removed label smoothing to stabilize loss
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=5e-3)  # Lower LR, higher weight decay
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Simple learning rate decay

scaler = torch.cuda.amp.GradScaler()  # Mixed precision training

# ================================
# 7️⃣ TRAINING LOOP (Loss Reduction)
# ================================
EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        with torch.amp.autocast("cuda"):  # Updated mixed precision format
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        correct += (outputs.argmax(dim=1) == labels).sum().item()
        total += labels.size(0)

    scheduler.step()
    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {running_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

# ================================
# 8️⃣ EVALUATION (Final)
# ================================
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        with torch.amp.autocast("cuda"):  # Updated mixed precision format
            outputs = model(inputs)

        correct += (outputs.argmax(dim=1) == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


C:\Users\vivek\AppData\Local\Temp\ipykernel_8228\1138976827.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Mixed precision training


Epoch 1/20 - Loss: 9.7425, Train Accuracy: 98.41%
Epoch 2/20 - Loss: 9.4619, Train Accuracy: 98.18%
Epoch 3/20 - Loss: 9.4702, Train Accuracy: 98.41%
Epoch 4/20 - Loss: 10.9902, Train Accuracy: 97.92%
Epoch 5/20 - Loss: 9.7471, Train Accuracy: 97.99%
Epoch 6/20 - Loss: 6.9767, Train Accuracy: 98.33%
Epoch 7/20 - Loss: 3.5183, Train Accuracy: 99.32%
Epoch 8/20 - Loss: 3.2301, Train Accuracy: 99.51%
Epoch 9/20 - Loss: 3.1356, Train Accuracy: 99.32%
Epoch 10/20 - Loss: 3.6473, Train Accuracy: 99.28%
Epoch 11/20 - Loss: 2.8020, Train Accuracy: 99.51%
Epoch 12/20 - Loss: 2.0400, Train Accuracy: 99.66%
Epoch 13/20 - Loss: 2.2708, Train Accuracy: 99.43%
Epoch 14/20 - Loss: 1.6932, Train Accuracy: 99.62%
Epoch 15/20 - Loss: 1.8623, Train Accuracy: 99.55%
Epoch 16/20 - Loss: 1.6428, Train Accuracy: 99.62%
Epoch 17/20 - Loss: 1.6257, Train Accuracy: 99.66%
Epoch 18/20 - Loss: 1.6313, Train Accuracy: 99.55%
Epoch 19/20 - Loss: 1.6909, Train Accuracy: 99.58%
Epoch 20/20 - Loss: 1.6494, Train Accur

In [5]:
torch.save(model.state_dict(), "densenet121_dermatology.pth")
print("✅ Model saved successfully!")

✅ Model saved successfully!
